In [1]:
import pandas as pd
import numpy as np
from scipy.linalg import hankel

import tensorflow as tf 
from tensorflow import keras
from keras.layers import LSTM, Dropout, Dense

2025-01-16 22:09:10.997505: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-16 22:09:11.007204: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737054551.018256  801922 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737054551.021407  801922 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-16 22:09:11.033129: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
print(len(tf.config.list_physical_devices('GPU')))

1


In [3]:
from lempel_ziv import *
from huffman import *
from utils import *

In [5]:
data = pd.read_csv('data/transact_18_22.csv', parse_dates=[2])
data.head(3)

,client,card,date,amt,mcc,group,value
0,390989,3048567,2018-01-01,878.0,5814,food,survival
1,1309946,3226559,2018-01-01,249.0,5411,food,survival
2,1386278,2715990,2018-01-01,50.0,5499,food,survival


In [6]:
filtered_data = data[(data['date'] >= '2019-01-15') & (data['date'] <= '2019-12-14')]
values_to_keep = ['socialization', 'survival', 'money', 'self_realization']
filtered_data = filtered_data[filtered_data['value'].isin(values_to_keep)]
filtered_data.head(3)

,client,card,date,amt,mcc,group,value
3278420,1378858,2391344,2019-01-15,350.87,5499,food,survival
3278421,3103680,3187433,2019-01-15,2660.00,5331,nonfood,socialization
3278422,2136426,3762147,2019-01-15,485.00,5331,nonfood,socialization


In [6]:
grouped = filtered_data.groupby(['client', 'date'])
new_data = []
for (client, date), group in grouped:
    row = {'client': client, 'date': date}
    for val in values_to_keep:
        row[val] = (int(np.round(np.log10(np.sum(group['amt']))))) if val in group['value'].values else 0  
    new_data.append(row)

new_df = pd.DataFrame(new_data)

In [21]:
new_df['code'] = new_df[['socialization', 'survival', 'money', 'self_realization']].apply(encode_row, axis=1)
new_df

,client,date,socialization,survival,money,self_realization,code
0,224,2019-01-15,2,0,0,0,4222
1,224,2019-01-16,3,3,0,3,5525
2,224,2019-01-17,3,3,0,0,5522
3,224,2019-01-18,2,0,0,0,4222
4,224,2019-01-19,3,0,0,3,5225
...,...,...,...,...,...,...,...
1588462,3564569,2019-12-10,0,3,0,3,2525
1588463,3564569,2019-12-11,3,3,0,0,5522
1588464,3564569,2019-12-12,0,3,0,0,2522
1588465,3564569,2019-12-13,0,3,0,3,2525


In [22]:
len(new_df['code'].unique())

91

In [24]:

alphabet = [
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
    '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '-', '_', '+', '=', '[', ']',  ';', ':', "'", '"', ',', '<', '.', '>', '/', '?', '|', '~', '`'
]
alphabet = { key: 0 for key in alphabet}

def get_code(code):
    global alphabet
    for key in alphabet.keys():
        if alphabet[key] == 0:
            alphabet[key] = code
        if alphabet[key] == code:
            return key

In [7]:
new_df['code'] = new_df['code'].apply(get_code)
new_df

NameError: name 'new_df' is not defined

In [26]:
clients_counts = new_df.groupby('client').size().reset_index(name='transaction_count')
sorted_clients = clients_counts.sort_values('transaction_count', ascending=False)
clients_counts = sorted_clients.head(1000)
new_df = new_df[new_df['client'].isin(clients_counts['client'])]
new_df.to_csv('output_with_log.csv')
new_df

,client,date,socialization,survival,money,self_realization,code
0,224,2019-01-15,2,0,0,0,0
1,224,2019-01-16,3,3,0,3,1
2,224,2019-01-17,3,3,0,0,2
3,224,2019-01-18,2,0,0,0,0
4,224,2019-01-19,3,0,0,3,3
...,...,...,...,...,...,...,...
1588462,3564569,2019-12-10,0,3,0,3,6
1588463,3564569,2019-12-11,3,3,0,0,2
1588464,3564569,2019-12-12,0,3,0,0,4
1588465,3564569,2019-12-13,0,3,0,3,6


In [4]:
new_df = pd.read_csv('output_with_log.csv', index_col=0)

In [5]:
selected_columns = ['date', 'socialization', 'survival', 'money', 'self_realization', 'code']

In [6]:
split = 52
lzw_depth = 56
forecast_horizons = [7]
start = 0

In [7]:
clients = list(new_df['client'].value_counts().index)

In [8]:
results_df = pd.DataFrame(
        columns=['client_id'] + [f'hp_{horizon}' for horizon in forecast_horizons] + ['huffman_complexity', 'lz_complexity']
)
results_df['client_id'] = clients  
results_df.head(5)

,client_id,hp_7,huffman_complexity,lz_complexity
0,2190618,NaN,NaN,NaN
1,208625,NaN,NaN,NaN
2,3441340,NaN,NaN,NaN
3,3483687,NaN,NaN,NaN
4,1421807,NaN,NaN,NaN


In [9]:
clients = list(new_df['client'].value_counts().index)
num_clients = len(clients)

In [10]:
train_days = 180

In [11]:
models = [keras.Sequential([
                LSTM(28, return_sequences=True),
                Dropout(0.3),
                LSTM(28, return_sequences=False),
                Dense(28, activation='tanh'),
                Dense(forecast_horizons[i], activation='sigmoid')
            ]) for i in range(len(forecast_horizons))]

for i in range(len(forecast_horizons)):
  models[i].compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

I0000 00:00:1737054574.135491  801922 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 36 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070, pci bus id: 0000:01:00.0, compute capability: 8.9
2025-01-16 22:09:34.147347: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1193] failed to allocate 36.75MiB (38535168 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory


In [1]:
with tf.device('/GPU:0'):
    for i in range(start, start + num_clients):
        client_transactions = new_df[new_df['client'] == clients[i]].loc[:, selected_columns]

        text = client_transactions['code'].values
        huffman_codes = huffman_code(text)
        encoded_string = ''.join(huffman_codes[symbol] for symbol in text)
        complexity = len(encoded_string) / len(text) / 4.0

        text = client_transactions['socialization'].values
        lzc = lempel_ziv_complexity(text)

        train_data = text[:train_days]
        test_data = text[train_days:]

        '''for j, forecast_horizon in enumerate(forecast_horizons):
            X_train, y_train = create_dataset(train_data, lzc, forecast_horizon)
            X_test, y_test = create_dataset(test_data, lzc, forecast_horizon)

            models[j].fit(X_train, y_train, epochs=15, batch_size=1, verbose=0)
            f1_scores, hp_scores = [], []

            for k in range(y_test.shape[0] - 1):
                predictions = models[j].predict(X_test[k:k+1], verbose=0)
                f1, accuracy = calculate_f1_accuracy(predictions.flatten().round(0).astype(int), y_test[k].flatten())
                f1_scores.append(f1)

            if len(f1_scores) > 0:
                hp_score = np.mean(np.array(f1_scores) > 0.75)
            else:
                hp_score = 0.0

            results_df.loc[results_df['client_id'] == clients[i], f'hp_{forecast_horizon}'] = hp_score
            results_df.loc[results_df['client_id'] == clients[i], 'huffman_complexity'] = complexity
            results_df.loc[results_df['client_id'] == clients[i], 'lz_complexity'] = lzc
        '''
        if i % 5 == 0:
            results_df.to_csv('out_log_1.csv')

NameError: name 'tf' is not defined